In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession 
from pyspark.conf import SparkConf
from pyspark import SparkContext


In [3]:
pyspark = SparkSession.builder \
.master('local[4]') \
.appName("Creating-RDD") \
.config("spark.executor.memory", "4g") \
.config("spark.driver.memory","2g") \
.getOrCreate()

In [4]:
sc = pyspark.sparkContext

# TEK RDD UZERINDE BASIC TRANSFORMATIONS


* map()
* filter()
* flatMap()
* distinct()


### map()

Her bir elemente bir fonksiyon uygular ve sonucunda yeni bbir RDD dondurur.

[1,2,3,4,5].map(lambda x : x*x) --> [1,4,9,16,25]


### filter()

Her bir elementi bir filtreden gecirir boolean degerlerden olusan yeni bir rdd dondurur.

[1,2,3,4,5].filter(lambda x : x >3) ---> [False,False,False,True,True] --> [4,5] 
! Sadece True olanlari sonuc olarakdondurur.

### map() & flatMap()

* map() sati bazinda islem yapar flatMap ise her ir satirinin her elemanina ayri ayri islem yapar. 

* map() --> 1'e 1 iliski varsa
* flatmap --> 1'den fazl iliski varsa word_count gibi 

In [5]:
RDD = sc.parallelize( [('Emel eve gel') ,('Ali ata bak'),('Ahmet okula git')])

In [6]:
RDD.take(2)

['Emel eve gel', 'Ali ata bak']

In [7]:
RDD_map_output = RDD.map(lambda x : x.upper())
RDD_map_output.take(5)

['EMEL EVE GEL', 'ALI ATA BAK', 'AHMET OKULA GIT']

In [8]:
RDD_flatMap_output = RDD.flatMap(lambda x : x.split(' '))
RDD_flatMap_output.take(7)

['Emel', 'eve', 'gel', 'Ali', 'ata', 'bak', 'Ahmet']

In [9]:
RDD_flatMap_output.map(lambda x:x.upper())
RDD_flatMap_output.take(2)

['Emel', 'eve']

Not : flatMap kelime kelime islem yapiyor. map satir satir islem yapiyor.

### distinct()

Tekrar eden elemanlari cikartiyor.

Pandas taki unique() 

In [12]:
RDD2 = sc.parallelize(['kahve','cay','kahve','seker','sut','sut'])

In [13]:
RDD2.take(3)

['kahve', 'cay', 'kahve']

In [14]:
RDD2_distinct_output = RDD2.distinct()

In [15]:
RDD2_distinct_output.take(3)

['kahve', 'cay', 'seker']

# IKI RDD BASIC TRANSFORMATIONS 

* union()
* intersection()
* subtract()
* cartesian()

Kumer operasyonlari


### union() -- birlestirme

rdd1.union(rdd2) --> iki rdd'yi oldugu gibi birlestirir. Tekrarli degerler olabilir.

### intersection() -- kesisim

rdd1.intersection(rdd2) --> iki rdd nin kesisimkumesini doner.

### subtract() -- fark

rdd1.subtract(rdd2) --> sadce rdd1 de bulunan elemanlari doner. 

### cartesian()
rdd1 ve rdd2 elemanlarinin tum ikili kombinasyonlarini dondurur.


# TEK RDD UZERINDE BASIC ACTIONS

* collect() --> tum elemanlari driver bilgisayara dondurur array olarak buyuk verilerde riskli 

* count() --> eleman sayisini dondurur. pandas taki len()

* countByValue() -->her bir elemanin rdd de kac kez gectigi bilgisini tupple olarak doner. (value_counts())

* take() -->rdd den istenen kadar degeri getirir. take(3)  (pandas taki head())

* top() --> top(3) en ustteki 3 elemani getirir.

* takeOrdered() --> elemanlari siralayarak belirtilen miktardaki lemani bir array olarak doner

* takeSample() --> elemanlardan bir orneklem aliyor 3 parametresi var
        * withReplacement :Boolean -->false dersek sectigi elemani bir daha secemez
        * num : Int --> kac eleman alacagini belirtir.
        * seed : Long --> random_state gibi
* reduce() --> rdd.reduce(lambda x,y --> x+y) 
iki degisken alir

* fold() --> reduce ile ayni islemi yapar sadece farkli olarak 0 degeri vardir.

* aggregate() --> rdd.aggregate(0)(seqOp,combOp)

* foreach() --> 


In [21]:
# sc.parallelize(['Ali',2,4,6,3,45,6,8,9,45,3,3,5,65,4,6])
RDD_numbers = sc.parallelize([2,4,6,3,45,6,8,9,45,3,3,5,65,4,6])

In [22]:
RDD_numbers.collect()

[2, 4, 6, 3, 45, 6, 8, 9, 45, 3, 3, 5, 65, 4, 6]

In [23]:
RDD_numbers.top(3)

[65, 45, 45]

In [24]:
RDD_names = sc.parallelize(['Ali','Beyza','Yeliz','Filiz','Cemre','Mina'])

In [25]:
RDD_names.top(3)

['Yeliz', 'Mina', 'Filiz']

Not : top siralanmis bir rdd uzerinden ilk 3 degeri dondurur ama string ve int degerirler var ise hata verir

Not : Alfabetik siraya gore tersten baslayarak dondurur yani once z sona y vs seklind

RDD_names.takeSample(False,3,42)

In [27]:
RDD_names.takeSample(True,3,42) # True yapinca getirdigi elemani tekrar tekrar secebiliyor.

['Filiz', 'Mina', 'Filiz']

In [30]:
RDD_names.takeOrdered(5)

['Ali', 'Beyza', 'Cemre', 'Filiz', 'Mina']

In [ ]:
RDD_numbers = sc.parallelize([2,4,6,3,45,6,8,9,45,3,3,5,65,4,6])
RDD_reduce_numbers =RDD_numbers.reduce(lambda x,y : x+y) 
RDD_reduce_numbers

In [40]:
RDD_numbers.fold(3, lambda x,y : x+y) # ????????? burada ki 0 degerini 1 ,2 ,3 yapinca degisen ne oluyor 

229

In [33]:
RDD_numbers.sum()

214

In [38]:
RDD_numbers.aggregate( (0,0), (lambda acc,value :(acc[0]+value,acc[1]+1)),(lambda acc1, acc2: (acc1[0]+acc2[0], acc1[1]+acc2[1])))

(214, 15)

In [39]:
RDD_numbers.aggregate( (0,0), (lambda x,y :(x[0]+y,x[1]+1)),(lambda x, y: (x[0]+y[0], x[1]+y[1])))

(214, 15)

In [3]:
aggreagte  ???? acumulator ??????????

SyntaxError: invalid syntax (<ipython-input-3-ee6803b89ac3>, line 1)

QUESTION &  ANSWERS

## Question 1

In [4]:
import findspark 
findspark.init()
from pyspark import SparkContext, SparkConf

In [7]:
spark_conf = SparkConf().\
setMaster('local[2]').\
setAppName("Test").\
set("spark.driver.mermory","2g").\
setExecutorEnv("spark.executer.memory","3g")

sc = SparkContext(conf=spark_conf)

In [8]:
print('Hello Spark')

Hello Spark


## Question 2 

In [9]:
rdd = sc.parallelize([3,4,5,76,3,54,6,8,4,3,6,8,9,97])

In [10]:
rdd.collect()

[3, 4, 5, 76, 3, 54, 6, 8, 4, 3, 6, 8, 9, 97]

### Question 3

In [18]:
text_rdd = sc.parallelize(["Learning spark is so exciting."])

In [19]:
text_rdd.map(lambda x : x.upper()).collect()

['LEARNING SPARK IS SO EXCITING.']

### Question 4

In [21]:
## line counts
text_rdd.count()

1

### Question 5  - 6- 7

In [23]:
## Word counts
text_rdd.flatMap(lambda x : x.split(" ")).count()

5

In [24]:
rdd_2 = sc.parallelize([1,2,3,4, 3, 54, 6, 8, 4,])

In [26]:
#set union

rdd.union(rdd_2).collect()

[3, 4, 5, 76, 3, 54, 6, 8, 4, 3, 6, 8, 9, 97, 1, 2, 3, 4, 3, 54, 6, 8, 4]

In [30]:
## set intersection
rdd.intersection(rdd_2).collect()

[4, 8, 54, 6, 3]

In [39]:
rdd_2_unique = rdd_2.distinct()
rdd_2_unique.collect()

[2, 4, 54, 6, 8, 1, 3]

In [38]:
# or 
numbers = [2,3,4,3,4,5,6,7,8,4,5,6,7,6,6]
rdd_3_set = set(numbers)
rdd_3_set

{2, 3, 4, 5, 6, 7, 8}

### Question 8

In [42]:
rdd_2.map(lambda x : (x,1)).collect().reduceByKey()

[(1, 1), (2, 1), (3, 1), (4, 1), (3, 1), (54, 1), (6, 1), (8, 1), (4, 1)]

In [44]:
rdd_2.map(lambda x : (x,1)).reduceByKey(lambda x,y : x+y).collect()

[(2, 1), (4, 2), (54, 1), (6, 1), (8, 1), (1, 1), (3, 2)]